## Import Libraries:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
# from sklearn import cross_validation, metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix ,classification_report,f1_score, roc_auc_score

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [ ]:
train = pd.read_csv("processed_data.csv")

In [ ]:
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]


In [ ]:
train

,A1,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19,A2,A20,A21,A22,A23,A24,A25,A26,A27,A28,A29,A3,A30,A4,A5,A6,A7,A8,A9,default
0,10.690000,1.786000,84.290000,13.460000,16.010000,52.180000,123101.840000,0.230000,4305.850000,0.150000,0.090000,86.100000,1.090000,10,0.010000,0.010000,0.010000,0.130000,0.000000,0.000000,0.000000,0.000000,8920.160000,0.000000,19912.850000,30.420000,31.720000,1.180000,145.410000,2.236000,0
1,28.500000,1.000000,59.800000,8.180000,29.330000,63.550000,34275.730000,0.800000,3616.670000,0.780000,0.140000,65.190000,1.670000,35,0.060000,0.010000,0.000000,0.090000,285.800000,0.000000,16.440000,1.980000,6564.000000,42.260000,6716.670000,32.500000,23.400000,2.740000,91.110000,1.000000,0
2,24.870000,1.400000,77.170000,10.250000,39.920000,61.950000,84841.790000,0.780000,2301.540000,0.620000,0.150000,73.790000,2.800000,31,0.080000,0.020000,0.020000,0.440000,494.620000,135.410000,127.450000,46.000000,4285.470000,14.920000,6463.750000,40.500000,51.530000,2.840000,93.900000,2.400000,0
3,13.640000,2.167000,95.880000,15.380000,49.800000,67.220000,145261.110000,0.090000,3419.600000,0.090000,0.000000,85.820000,1.000000,29,0.030000,0.010000,0.000000,0.050000,1015.190000,0.000000,210.630000,92.580000,6887.560000,0.000000,9244.440000,49.560000,53.440000,0.360000,171.450000,1.875000,0
4,4.500000,1.500000,94.550000,11.870000,17.430000,50.700000,120522.800000,0.170000,3828.570000,0.080000,0.000000,95.920000,2.690000,22,0.010000,0.000000,0.000000,0.040000,0.000000,106.410000,79.000000,0.000000,8746.500000,0.000000,19987.500000,32.750000,37.880000,0.920000,153.250000,2.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179991,20.074244,1.156038,63.778475,13.218483,19.449733,72.524720,78615.326179,0.301217,4380.997951,0.397718,0.100609,68.169026,1.452364,15,0.020000,0.000000,0.000152,0.070456,90.708018,5.783996,99.212400,23.497874,4219.465747,1.486392,6255.636721,39.875703,54.792027,2.727940,93.478926,1.452901,1
179992,25.547752,0.784095,60.440393,10.333042,49.185891,74.435900,58930.695777,0.703930,4357.424419,0.433289,0.119095,72.359241,1.950508,90,0.074260,0.000839,0.011678,0.071744,14.559422,923.917771,20.970626,4.772428,3591.554531,0.000000,7084.208192,60.021024,67.084821,2.809405,106.114756,0.949669,1
179993,32.478535,0.582875,62.052127,7.441100,24.234672,66.375770,73937.640032,0.676767,2212.057355,0.488102,0.225044,64.787632,1.485818,69,0.046854,0.004095,0.008190,0.082285,47.492465,644.612669,122.708766,26.321687,4729.198348,0.000000,5882.696922,46.502621,52.098201,2.108227,63.247374,0.644298,1
179994,15.614411,1.268208,82.954814,11.076296,25.029126,57.812556,65647.487251,0.334468,3710.724141,0.255851,0.084830,81.300550,1.264510,85,0.026553,0.000000,0.000000,0.063447,18.880022,530.280739,48.121396,48.343527,6446.126736,3.884580,9568.271119,39.813967,45.143784,2.259062,128.174258,1.352582,1


## Baseline Model

Since here the criteria is AUC, simply predicting the most prominent class would give an AUC of 0.5 always. Another way of getting a baseline model is to use the algorithm without tuning, i.e. with default parameters.

In [ ]:
x=train[train.columns.difference(['default'])]
y=train['default']

In [ ]:
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(x, y)

GradientBoostingClassifier(random_state=10)

In [ ]:
y_train_reg_gb = gbm0.predict(x)

train_accuracy_reg_gb = accuracy_score(y, y_train_reg_gb)*100

print('Confusion matrix:\n', confusion_matrix(y, y_train_reg_gb))
print('f1 score:', (f1_score(y, y_train_reg_gb))*100)
print('Training accuracy: %.4f %%' % train_accuracy_reg_gb)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gbm0.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[85402  4596]
 [ 8521 81477]]
f1 score: 92.5501644223069
Training accuracy: 92.7126 %
AUC Scorer: 97.7934 %


## GBM Models:

There 2 types of parameters here:
1. Tree-specific parameters
  * min_samples_split
  * min_samples_leaf
  * max_depth
  * min_leaf_nodes
  * max_features
  * loss function
2. Boosting specific paramters
  * n_estimators
  * learning_rate
  * subsample

## Approach for tackling the problem

1. Decide a relatively higher value for learning rate and tune the number of estimators requried for that.
2. Tune the tree specific parameters for that learning rate
3. Tune subsample
4. Lower learning rate as much as possible computationally and increase the number of estimators accordingly.

## Step 1- Find the number of estimators for a high learning rate

We will use the following benchmarks for parameters:
1. min_samples_split = 500 : ~0.5-1% of total values. Since this is imbalanced class problem, we'll take small value
2. min_samples_leaf = 50 : Just using for preventing overfitting. will be tuned later.
3. max_depth = 8 : since high number of observations and predictors, choose relatively high value
4. max_features = 'sqrt' : general thumbrule to start with
5. subsample = 0.8 : typically used value (will be tuned later)

0.1 is assumed to be a good learning rate to start with. Let's try to find the optimum number of estimators requried for this.

In [ ]:
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=1, min_samples_split=500,
                                  min_samples_leaf=50,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
gsearch1.fit(x,y)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=1, max_depth=8,
                                                  max_features='sqrt',
                                                  min_samples_leaf=50,
                                                  min_samples_split=500,
                                                  random_state=10,
                                                  subsample=0.8),
             n_jobs=4, param_grid={'n_estimators': range(20, 81, 10)},
             scoring='roc_auc')

In [ ]:
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

({'mean_fit_time': array([ 42.4550271 ,  59.99850807,  79.16236   ,  99.00966983,
         118.37824421, 140.14121027, 135.03234735]),
  'mean_score_time': array([0.15806098, 0.19625883, 0.23148055, 0.26884723, 0.2978982 ,
         0.33926125, 0.27354517]),
  'mean_test_score': array([0.98277967, 0.98474645, 0.98564829, 0.9862869 , 0.98660695,
         0.9868366 , 0.98674727]),
  'param_n_estimators': masked_array(data=[20, 30, 40, 50, 60, 70, 80],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'n_estimators': 20},
   {'n_estimators': 30},
   {'n_estimators': 40},
   {'n_estimators': 50},
   {'n_estimators': 60},
   {'n_estimators': 70},
   {'n_estimators': 80}],
  'rank_test_score': array([7, 6, 5, 4, 3, 1, 2], dtype=int32),
  'split0_test_score': array([0.93351892, 0.93543363, 0.93857011, 0.9406704 , 0.9418339 ,
         0.94289653, 0.94321014]),
  'split1_test_score': array([0.9948853 , 0.996

1.45 ->40
0.04->80
0.8->80
I got 40 as the optimal estimators for the 0.1 learning rate. So i reduced the LR to 0.07 that gave me optimal estimators of 60 which seemed to tbe reasonable

## Step 2- Tune tree-specific parameters
Now, lets move onto tuning the tree parameters. We will do this in 2 stages:
1. Tune max_depth and num_samples_split
2. Tune min_samples_leaf
3. Tune max_features

In [ ]:
#Grid seach on subsample and max_features
param_test2 = {'max_depth':range(5,16,2), 'min_samples_split':range(200,1001,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=1, n_estimators=70,
                                                max_features='sqrt', subsample=0.8, random_state=10), 
                       param_grid = param_test2, scoring='roc_auc',n_jobs=4, cv=5)
gsearch2.fit(x,y)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=1,
                                                  max_features='sqrt',
                                                  n_estimators=70,
                                                  random_state=10,
                                                  subsample=0.8),
             n_jobs=4,
             param_grid={'max_depth': range(5, 16, 2),
                         'min_samples_split': range(200, 1001, 200)},
             scoring='roc_auc')

In [ ]:
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

({'mean_fit_time': array([ 92.90082378,  91.85219693,  93.36993227,  91.20075254,
          90.67240353, 123.70337801, 123.75519924, 124.44054251,
         124.95570984, 122.79872766, 155.23668356, 156.40516644,
         156.76788559, 155.34143047, 155.69791875, 192.08146992,
         188.67009039, 188.11520009, 186.52264442, 182.71242666,
         218.52028031, 218.33945246, 217.05910058, 215.30705547,
         214.62192769, 249.61295323, 245.71999598, 244.17184825,
         242.01792936, 204.45761032]),
  'mean_score_time': array([0.26861601, 0.26048779, 0.25356092, 0.24765329, 0.25775256,
         0.28976722, 0.28661852, 0.30015869, 0.29289255, 0.28761215,
         0.33719745, 0.31458554, 0.33493981, 0.33023701, 0.33763447,
         0.36214356, 0.36366768, 0.37460093, 0.36963701, 0.38522754,
         0.39894438, 0.40774713, 0.41215   , 0.41433558, 0.39427018,
         0.46892128, 0.45607095, 0.45166945, 0.43541007, 0.33019042]),
  'mean_test_score': array([0.98243159, 0.9842754 , 0.

Since we reached the maximum of min_sales_split, we should check higher values as well. Also, we can tune min_samples_leaf with it now as max_depth is fixed. One might argue that max depth might change for higher value but if you observe the output closely, a max_depth of 9 had a better model for most of cases.
So lets perform a grid search on them:

In [ ]:
#Grid seach on subsample and max_features
param_test3 = {'min_samples_split':range(1000,2100,200), 'min_samples_leaf':range(30,71,10)}
gsearch3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=1, n_estimators=70,max_depth=5,
                                                    max_features='sqrt', subsample=0.8, random_state=10), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4, cv=5)
gsearch3.fit(x,y)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=1, max_depth=5,
                                                  max_features='sqrt',
                                                  n_estimators=70,
                                                  random_state=10,
                                                  subsample=0.8),
             n_jobs=4,
             param_grid={'min_samples_leaf': range(30, 71, 10),
                         'min_samples_split': range(1000, 2100, 200)},
             scoring='roc_auc')

In [ ]:
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

({'mean_fit_time': array([90.30625596, 90.07436347, 90.41050148, 89.62626534, 90.07039809,
         91.07661333, 90.52938418, 90.64863582, 89.90032396, 89.73792934,
         90.46584764, 89.98852997, 89.5994905 , 89.97073236, 90.10009274,
         90.39328504, 90.44121585, 89.88053145, 90.37105689, 91.52765636,
         91.36811619, 90.47781816, 89.98160257, 89.83703303, 90.36206779,
         90.3749752 , 90.77912712, 90.69452896, 90.09127727, 76.71915727]),
  'mean_score_time': array([0.25431871, 0.25243902, 0.24467311, 0.2471025 , 0.25151153,
         0.25355706, 0.25212641, 0.26195946, 0.2479527 , 0.2354877 ,
         0.24653711, 0.24752517, 0.24945493, 0.24888453, 0.26016579,
         0.24883504, 0.25351348, 0.25562878, 0.25621066, 0.25832529,
         0.25168939, 0.24780703, 0.24515405, 0.25798006, 0.25129652,
         0.2473525 , 0.25904961, 0.24594212, 0.2460062 , 0.18544254]),
  'mean_test_score': array([0.98407743, 0.98392568, 0.98402848, 0.98400696, 0.98433432,
         0.984

In [ ]:
res_gsearch3 = gsearch3.predict(x)

train_accuracy_gsearch3 = accuracy_score(y, res_gsearch3)*100

print('Confusion matrix:\n', confusion_matrix(y, res_gsearch3))
print('f1 score:', (f1_score(y, res_gsearch3))*100)
print('Training accuracy: %.4f %%' % train_accuracy_gsearch3)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gsearch3.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[88328  1670]
 [ 4013 85985]]
f1 score: 96.80106724907544
Training accuracy: 96.8427 %
AUC Scorer: 99.0783 %


Tune max_features:

In [ ]:
#Grid seach on subsample and max_features
param_test4 = {'max_features':range(7,20,2)}
gsearch4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=1, n_estimators=70,max_depth=5, 
                            min_samples_split=1800, min_samples_leaf=50, subsample=0.8, random_state=10),
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4, cv=5)
gsearch4.fit(x,y)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=1, max_depth=5,
                                                  min_samples_leaf=50,
                                                  min_samples_split=1800,
                                                  n_estimators=70,
                                                  random_state=10,
                                                  subsample=0.8),
             n_jobs=4, param_grid={'max_features': range(7, 20, 2)},
             scoring='roc_auc')

In [ ]:
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

({'mean_fit_time': array([120.39386735, 151.50477939, 181.63233943, 210.02233739,
         240.42621427, 272.19685216, 257.96962075]),
  'mean_score_time': array([0.25357976, 0.2397222 , 0.23079491, 0.23132143, 0.22603569,
         0.23032122, 0.15318232]),
  'mean_test_score': array([0.98395334, 0.98383975, 0.98508035, 0.9846032 , 0.98530501,
         0.98471555, 0.98453025]),
  'param_max_features': masked_array(data=[7, 9, 11, 13, 15, 17, 19],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_features': 7},
   {'max_features': 9},
   {'max_features': 11},
   {'max_features': 13},
   {'max_features': 15},
   {'max_features': 17},
   {'max_features': 19}],
  'rank_test_score': array([6, 7, 2, 4, 1, 3, 5], dtype=int32),
  'split0_test_score': array([0.92705154, 0.92529172, 0.93128081, 0.92850921, 0.93103708,
         0.92864683, 0.92659465]),
  'split1_test_score': array([0.99816054, 0.9983324

## Step3- Tune Subsample and Lower Learning Rate

In [ ]:
#Grid seach on subsample and max_features
param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9]}
gsearch5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=1, n_estimators=70,max_depth=5, 
                            min_samples_split=1800, min_samples_leaf=50, subsample=0.8, random_state=10, max_features=15),
                       param_grid = param_test5, scoring='roc_auc',n_jobs=4, cv=5)
gsearch5.fit(x,y)

GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=1, max_depth=5,
                                                  max_features=15,
                                                  min_samples_leaf=50,
                                                  min_samples_split=1800,
                                                  n_estimators=70,
                                                  random_state=10,
                                                  subsample=0.8),
             n_jobs=4,
             param_grid={'subsample': [0.6, 0.7, 0.75, 0.8, 0.85, 0.9]},
             scoring='roc_auc')

In [ ]:
gsearch5.cv_results_, gsearch5.best_params_, gsearch5.best_score_

({'mean_fit_time': array([175.82715249, 203.1861403 , 215.36270218, 228.21957226,
         241.00725751, 208.71028109]),
  'mean_score_time': array([0.21664329, 0.21950545, 0.21635356, 0.22115493, 0.21902442,
         0.16433578]),
  'mean_test_score': array([0.98433868, 0.98535786, 0.98490075, 0.98530501, 0.98475175,
         0.98468182]),
  'param_subsample': masked_array(data=[0.6, 0.7, 0.75, 0.8, 0.85, 0.9],
               mask=[False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'subsample': 0.6},
   {'subsample': 0.7},
   {'subsample': 0.75},
   {'subsample': 0.8},
   {'subsample': 0.85},
   {'subsample': 0.9}],
  'rank_test_score': array([6, 1, 3, 2, 4, 5], dtype=int32),
  'split0_test_score': array([0.92735912, 0.93192782, 0.92923978, 0.93103708, 0.92767667,
         0.92734795]),
  'split1_test_score': array([0.99853908, 0.99857468, 0.99853011, 0.99855535, 0.99888121,
         0.99904019]),
  'split2_test_score': array

With all tuned lets try reducing the learning rate and proportionally increasing the number of estimators to get more robust results:

In [ ]:
gbm_tuned_1 = GradientBoostingClassifier(learning_rate=0.8, n_estimators=140,max_depth=5, min_samples_split=1800, 
                                         min_samples_leaf=50, subsample=0.7, random_state=10, max_features=15)


gbm_tuned_1.fit(x,y)
res_gbm_tuned_1 = gbm_tuned_1.predict(x)

train_accuracy_gbm_tuned_1 = accuracy_score(y, res_gbm_tuned_1)*100

print('Confusion matrix:\n', confusion_matrix(y, res_gbm_tuned_1))
print('f1 score:', (f1_score(y, res_gbm_tuned_1))*100)
print('Training accuracy: %.4f %%' % train_accuracy_gbm_tuned_1)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gbm_tuned_1.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[89286   712]
 [ 3295 86703]]
f1 score: 97.74142819297347
Training accuracy: 97.7738 %
AUC Scorer: 99.3850 %


1/10th learning rate

In [ ]:
gbm_tuned_2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=600,max_depth=5, min_samples_split=1800, 
                                         min_samples_leaf=50, subsample=0.7, random_state=10, max_features=15)

gbm_tuned_2.fit(x,y)
res_gbm_tuned_2 = gbm_tuned_2.predict(x)

train_accuracy_gbm_tuned_2 = accuracy_score(y, res_gbm_tuned_2)*100

print('Confusion matrix:\n', confusion_matrix(y, res_gbm_tuned_2))
print('f1 score:', (f1_score(y, res_gbm_tuned_2))*100)
print('Training accuracy: %.4f %%' % train_accuracy_gbm_tuned_2)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gbm_tuned_2.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[89726   272]
 [ 3651 86347]]
f1 score: 97.77880951437291
Training accuracy: 97.8205 %
AUC Scorer: 99.4379 %


1/50th learning rate

In [ ]:
gbm_tuned_3 =  


gbm_tuned_3.fit(x,y)
res_gbm_tuned_3 = gbm_tuned_3.predict(x)

train_accuracy_gbm_tuned_3 = accuracy_score(y, res_gbm_tuned_3)*100

print('Confusion matrix:\n', confusion_matrix(y, res_gbm_tuned_3))
print('f1 score:', (f1_score(y, res_gbm_tuned_3))*100)
print('Training accuracy: %.4f %%' % train_accuracy_gbm_tuned_3)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gbm_tuned_3.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[89883   115]
 [ 3082 86916]]
f1 score: 98.19408119573629
Training accuracy: 98.2238 %
AUC Scorer: 99.6007 %


In [ ]:
#Choose all predictors except target & IDcols
gbm_tuned_4 = GradientBoostingClassifier(learning_rate=0.09, n_estimators=1500,max_depth=5, min_samples_split=1800, 
                                         min_samples_leaf=50, subsample=0.7, random_state=10, max_features=15,
                                         warm_start=True)


gbm_tuned_4.fit(x,y)
res_gbm_tuned_4 = gbm_tuned_4.predict(x)

train_accuracy_gbm_tuned_4 = accuracy_score(y, res_gbm_tuned_4)*100

print('Confusion matrix:\n', confusion_matrix(y, res_gbm_tuned_4))
print('f1 score:', (f1_score(y, res_gbm_tuned_4))*100)
print('Training accuracy: %.4f %%' % train_accuracy_gbm_tuned_4)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gbm_tuned_4.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[89925    73]
 [ 2772 87226]]
f1 score: 98.39534791902852
Training accuracy: 98.4194 %
AUC Scorer: 99.6838 %


In [ ]:
gbm_tuned_4 = GradientBoostingClassifier(learning_rate=0.08, n_estimators=1800,max_depth=5, min_samples_split=1800, 
                                         min_samples_leaf=50, subsample=0.7, random_state=10, max_features=15,
                                         warm_start=True)


gbm_tuned_4.fit(x,y)
res_gbm_tuned_4 = gbm_tuned_4.predict(x)

train_accuracy_gbm_tuned_4 = accuracy_score(y, res_gbm_tuned_4)*100

print('Confusion matrix:\n', confusion_matrix(y, res_gbm_tuned_4))
print('f1 score:', (f1_score(y, res_gbm_tuned_4))*100)
print('Training accuracy: %.4f %%' % train_accuracy_gbm_tuned_4)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gbm_tuned_4.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[89936    62]
 [ 2665 87333]]
f1 score: 98.46273528267744
Training accuracy: 98.4850 %
AUC Scorer: 99.7172 %


In [ ]:
gbm_tuned_4 = GradientBoostingClassifier(learning_rate=0.08, n_estimators=2000,max_depth=5, min_samples_split=1800, 
                                         min_samples_leaf=50, subsample=0.7, random_state=10, max_features=15,
                                         warm_start=True)


gbm_tuned_4.fit(x,y)
res_gbm_tuned_4 = gbm_tuned_4.predict(x)

train_accuracy_gbm_tuned_4 = accuracy_score(y, res_gbm_tuned_4)*100

print('Confusion matrix:\n', confusion_matrix(y, res_gbm_tuned_4))
print('f1 score:', (f1_score(y, res_gbm_tuned_4))*100)
print('Training accuracy: %.4f %%' % train_accuracy_gbm_tuned_4)
print('AUC Scorer: %.4f %%'% (roc_auc_score(y, gbm_tuned_4.predict_proba(x)[:, 1])*100))

Confusion matrix:
 [[89950    48]
 [ 2474 87524]]
f1 score: 98.57971504195528
Training accuracy: 98.5989 %
AUC Scorer: 99.7614 %
